In [1]:
#!/usr/bin/python

In [2]:
# coding: utf-8

In [3]:
import re
import fileinput
import subprocess
from IPython.display import Markdown, display
from urllib import quote_plus

GITHUB_USER="DivLoic"
BRACH_PATTERN=r"branch[=,/]([\w,%,-]*)[/,)]"

In [4]:
help(re.sub)

Help on function sub in module re:

sub(pattern, repl, string, count=0, flags=0)
    Return the string obtained by replacing the leftmost
    non-overlapping occurrences of the pattern in string by the
    replacement repl.  repl can be either a string or a callable;
    if a string, backslash escapes in it are processed.  If it is
    a callable, it's passed the match object and must return
    a replacement string to be used.



In [5]:
def applyBranch(line, branch_name, pattern="{{branch}}"):
    return line.replace(pattern, quote_plus(branch_name))

def unapplyBranch(line, branch_name, pattern="{{branch}}"):
    return line.replace(quote_plus(branch_name), pattern)

def extractBranch(line):
    return re.search(BRACH_PATTERN, line).group(1)

def checkoutBranch(line, pattern="{{branch}}"):
    branch = extractBranch(line)
    return line.replace(branch, pattern)
    
def editRmd(file_name, branch_name, func):
    stream = fileinput.FileInput(file_name, inplace=True)
    for line in stream:
        print func(line, branch_name),
        
    stream.close()
    
    

In [6]:
BRANCH = subprocess.check_output(["git", "rev-parse", "--abbrev-ref", "HEAD"]).strip()

In [7]:
BRANCH

'master'

In [8]:
subprocess.check_output(["git", "add", "README.md"])

''

### Test the hooks funcitons

In [9]:
import sys
import unittest
from unittest import TestCase

import __main__

In [10]:
class TestApplyBranch(unittest.TestCase):
    
    def test_setup_branch(self):
        # given 
        branch = "feature/everything-is-awsome"
        line = "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch={{branch}})]\
(https://travis-ci.org/DivLoic/kafka-connect-satori)"
        
        # when
        result = applyBranch(line, branch_name=branch)
        
        # then
        self.assertEqual(
            result, 
            "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch\
=feature%2Feverything-is-awsome)](https://travis-ci.org/DivLoic/kafka-connect-satori)")

In [11]:
class TestUnApplyBranch(unittest.TestCase):
    
    def test_delete_branch_param(self):
        # given 
        branch = "master"
        line1 = "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch=master)]\
(https://travis-ci.org/DivLoic/kafka-connect-satori)"
        
        # when
        result1 = unapplyBranch(line1, branch_name=branch)
        
        # then
        self.assertEqual(
            result1, 
            "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch\
={{branch}})](https://travis-ci.org/DivLoic/kafka-connect-satori)")
        
    def test_delete_branch_segment(self):
        # given
        branch = "master"
        line2 = "[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/branch/master/graph/badge.svg)]\
(https://codecov.io/gh/DivLoic/kafka-connect-satori)"
        
        # when
        result2 = unapplyBranch(line2, branch_name=branch)
        
        # then
        self.assertEqual(
            result2,
            "[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/branch/{{branch}}/graph/badge.svg)]\
(https://codecov.io/gh/DivLoic/kafka-connect-satori)")

In [12]:
class TestCheckoutBranch(unittest.TestCase):
    
    def test_extract_branch(self):
    
        # given 
        line1 = "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch=devel)]\
(https://travis-ci.org/DivLoic/kafka-connect-satori)"

        line2 = "[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/\
branch/feature2%Ffoo-bar/graph/badge.svg)](https://codecov.io/gh/DivLoic/kafka-connect-satori)"

        # when 
        result1 = extractBranch(line1)
        result2 = extractBranch(line2)

        # then
        self.assertEqual(result1, "devel")
        self.assertEqual(result2, "feature2%Ffoo-bar")
        
    
    def test_checkout_branch(self):
    
        # given 
        line1 = "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch=devel)]\
(https://travis-ci.org/DivLoic/kafka-connect-satori)"

        line2 = "[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/\
branch/feature2%Ffoo-bar/graph/badge.svg)](https://codecov.io/gh/DivLoic/kafka-connect-satori)"

        # when 
        result1 = checkoutBranch(line1)
        result2 = checkoutBranch(line2)

        # then
        self.assertEqual(result1, "[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?\
branch={{branch}})](https://travis-ci.org/DivLoic/kafka-connect-satori)")
        
        self.assertEqual(result2, "[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/\
branch/{{branch}}/graph/badge.svg)](https://codecov.io/gh/DivLoic/kafka-connect-satori)")
    

In [13]:
unittest.main(argv=['ignored', '-v'], exit=False);

test_setup_branch (__main__.TestApplyBranch) ... ok
test_checkout_branch (__main__.TestCheckoutBranch) ... ok
test_extract_branch (__main__.TestCheckoutBranch) ... ok
test_delete_branch_param (__main__.TestUnApplyBranch) ... ok
test_delete_branch_segment (__main__.TestUnApplyBranch) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


In [14]:
checkoutBranch("[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch=devel)]\
(https://travis-ci.org/DivLoic/kafka-connect-satori)")

'[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch={{branch}})](https://travis-ci.org/DivLoic/kafka-connect-satori)'

In [15]:
display(Markdown("[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch\
=devel)](https://travis-ci.org/DivLoic/kafka-connect-satori)"))
display(Markdown("[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/branch/devel/graph/badge.svg)]\
(https://codecov.io/gh/DivLoic/kafka-connect-satori)"))

[![Build Status](https://travis-ci.org/DivLoic/kafka-connect-satori.svg?branch=devel)](https://travis-ci.org/DivLoic/kafka-connect-satori)

[![codecov](https://codecov.io/gh/DivLoic/kafka-connect-satori/branch/devel/graph/badge.svg)](https://codecov.io/gh/DivLoic/kafka-connect-satori)

### Apply the hook functions

#### pre commit 

In [16]:
editRmd("README.md", BRANCH, applyBranch)

#### post commit

In [17]:
editRmd("README.md", BRANCH, unapplyBranch)

In [ ]:
#### post checkout

In [ ]:
editRmd("README.md", BRANCH, checkoutBranch)

In [18]:
emoji_ok = u'\xF0\x9F\x91\x8C'
emoji_book = u'\xF0\x9F\x93\x91'
emoji_memo = u'\xF0\x9F\x93\x9D'

#import re
#line = re.sub(r"branch[=|\/]()", "feauture/foo/bar" % fileIn, line)

In [19]:
print emoji_book
print emoji_memo
print emoji_ok

ð
ð
ð
